# 3. Predictive Text, Part I: N-Grams

In this section, we will be building predictive text, a system that suggests the next word while a user is typing. Predictive text is used in mobile phone keyboards, search applications, AI-powered email composition, and more.

The primary goal of a predictive text system is **given some sequence of words, what is the most likely next word?**

This is essentially the same problem as the NLP concept of a **Language Model**. A language model attempts to calculate the probability of a sequence of words. Creating good language models is a huge field of NLP, and many of the state-of-the-art language models today require enormous amounts of data. However, we typically do not have sufficient data for low-resource languages to use these sorts of models, so this tutorial will use models that are effective even with more limited corpora.

## Loading the corpus
For this application, we are using english text from the COCA corpus. We chose to use English rather than Uspanteko so that the reader can easily judge the goodness of the predictions. However, feel free to use your own preferred language, making the necessary changes when indicated.

In [1]:
# Load our data
import util
corpus = util.load_raw_text(corpus_directory="corpus-eng") # REPLACE WITH YOUR CORPUS FOLDER
corpus[:1000]

'\n@@17141 ERIC @!BURNS , FOX NEWS HOST : On this week \'s " FOX News Watch " : The president and the hurricane : will he weather the storm ? The media helping victims to weather the storm . How did the media judge the judge ? How did audiences judge Martha on her return to TV ? And everybody has to go to the bathroom sometime . We will cover the coverage of these stories right after the headlines . @(NEWSBREAK) @!BURNS : You \'ve heard of slow times in the news business ? The past few weeks have been as fast as they get . So we had better get started . Here are Jim Pinkerton of " Newsday " , syndicated columnist Cal Thomas , Jane Hall of the American University , and media writer Neal Gabler . I \'m Eric Burns . " FOX News Watch " is coming right up . @(BEGIN-VIDEO-CLIP) @!GEORGE-W-BUSH-PRE : Americans have every right to expect a more effective response in a time of emergency . When the federal government fails to meet such an obligation , I as president am responsible for the proble

Next, we need to tokenize our text. However, we want to keep punctuation that ends a sentence this time, since we care about dividing the text into sentences.

If you are using your own language, you will need to split the data into a list of words, including ending punctuation. You can do this with a tokenizing function based on the format of your data.

In [2]:
import re

def preprocess(text):
    text = util.strip_accents(text)
    text = text.lower()

    # Unlike our Uspanteko data, this is already separated by spaces, so we can just split by spaces (rather than tokenizing)
    # REPLACE WITH TOKENIZATION METHOD FOR YOUR LANGUAGE
    tokens = text.split(" ")

    tokens_filtered = []
    for token in tokens:
        # Skip any tokens with special characters
        if re.match(r"[\w|\']+|[\.|\,|\?|\!]", token):
            tokens_filtered.append(token)
    return tokens_filtered


tokens_filtered = preprocess(corpus)
print(len(tokens_filtered))
tokens_filtered[:100]

1041158


['eric',
 ',',
 'fox',
 'news',
 'host',
 'on',
 'this',
 'week',
 "'s",
 'fox',
 'news',
 'watch',
 'the',
 'president',
 'and',
 'the',
 'hurricane',
 'will',
 'he',
 'weather',
 'the',
 'storm',
 '?',
 'the',
 'media',
 'helping',
 'victims',
 'to',
 'weather',
 'the',
 'storm',
 '.',
 'how',
 'did',
 'the',
 'media',
 'judge',
 'the',
 'judge',
 '?',
 'how',
 'did',
 'audiences',
 'judge',
 'martha',
 'on',
 'her',
 'return',
 'to',
 'tv',
 '?',
 'and',
 'everybody',
 'has',
 'to',
 'go',
 'to',
 'the',
 'bathroom',
 'sometime',
 '.',
 'we',
 'will',
 'cover',
 'the',
 'coverage',
 'of',
 'these',
 'stories',
 'right',
 'after',
 'the',
 'headlines',
 '.',
 'you',
 "'ve",
 'heard',
 'of',
 'slow',
 'times',
 'in',
 'the',
 'news',
 'business',
 '?',
 'the',
 'past',
 'few',
 'weeks',
 'have',
 'been',
 'as',
 'fast',
 'as',
 'they',
 'get',
 '.',
 'so',
 'we',
 'had']

This corpus is much larger than most low-resource language corpora will be, so we will shorten it.

In [3]:
tokens_filtered = tokens_filtered[:50000]

## N-Grams
Now, let's build our first model. For this we will use *n-grams*.

An n-gram is simply a sequence of *n* words that occurs in our text. For instance, consider the sentence:

> What is your name?

If we got the list of all of the *bigrams* (2 words each) in the sentence, we would have:

- *What is*
- *is your*
- *your name*
- *name ?*

If we got the list of all of the *trigrams* (3 words each) we would get:

- *What is your*
- *is your name*
- *your name ?*

And so on, and so forth.

### Using N-grams as a language model

We can easily use the idea of n-grams to build a simple language model. For instance, if we are trying to predict the next word, using trigrams, given the input:

> What is your ...

We can look at all of the trigrams that start with *is your*, and choose the most common one. Let's use our tokenized text to get a list of all of the n-grams that occur in the text. 

**Note:** Right now, we would have n-grams that cross sentence boundaries, such as "headlines . you". This isn't super useful, so a common technique is to **pad** each sentence with tokens representing the start of the sentence.

In [4]:
# Pad each sentence with some number of start symbols 
def pad(text: list, num_padding: int):
    padded_text = []
    
    # Add initial padding to the first sentence
    for _ in range(num_padding):
        padded_text.append("<s>")
    
    for word in text:
        padded_text.append(word)

        # Every time we see an end punctuation mark, add <s> tokens before it
        # REPLACE IF YOUR LANGUAGE USES DIFFERENT END PUNCTUATION
        if word in [".", "?", "!"]:
            for _ in range(num_padding):
                padded_text.append("<s>")
        
        
    return padded_text

pad(tokens_filtered, 2)[:30]

['<s>',
 '<s>',
 'eric',
 ',',
 'fox',
 'news',
 'host',
 'on',
 'this',
 'week',
 "'s",
 'fox',
 'news',
 'watch',
 'the',
 'president',
 'and',
 'the',
 'hurricane',
 'will',
 'he',
 'weather',
 'the',
 'storm',
 '?',
 '<s>',
 '<s>',
 'the',
 'media',
 'helping']

In [5]:
from nltk.util import ngrams

# Now, we can actually create the list of n-grams using the NLTK library
padded_tokens = pad(tokens_filtered, 2)
trigrams = list(ngrams(sequence=padded_tokens, n=3))
trigrams[:30]

[('<s>', '<s>', 'eric'),
 ('<s>', 'eric', ','),
 ('eric', ',', 'fox'),
 (',', 'fox', 'news'),
 ('fox', 'news', 'host'),
 ('news', 'host', 'on'),
 ('host', 'on', 'this'),
 ('on', 'this', 'week'),
 ('this', 'week', "'s"),
 ('week', "'s", 'fox'),
 ("'s", 'fox', 'news'),
 ('fox', 'news', 'watch'),
 ('news', 'watch', 'the'),
 ('watch', 'the', 'president'),
 ('the', 'president', 'and'),
 ('president', 'and', 'the'),
 ('and', 'the', 'hurricane'),
 ('the', 'hurricane', 'will'),
 ('hurricane', 'will', 'he'),
 ('will', 'he', 'weather'),
 ('he', 'weather', 'the'),
 ('weather', 'the', 'storm'),
 ('the', 'storm', '?'),
 ('storm', '?', '<s>'),
 ('?', '<s>', '<s>'),
 ('<s>', '<s>', 'the'),
 ('<s>', 'the', 'media'),
 ('the', 'media', 'helping'),
 ('media', 'helping', 'victims'),
 ('helping', 'victims', 'to')]

Now that we have a list of trigrams, we can count up the frequency of each different trigram.

In [6]:
# Get a dict of all trigrams and their frequency
# This is the same as how we counted words in the spellchecker section
all_trigrams = dict()
for gram in trigrams:
    if gram in all_trigrams:
        all_trigrams[gram] += 1
    else:
        all_trigrams[gram] = 1
        
len(all_trigrams)

41845

In [7]:
# Let's see what the twenty most common trigrams are
sorted(all_trigrams.items(), key=lambda x: x[1], reverse=True)[:30]

[(('.', '<s>', '<s>'), 2379),
 (('?', '<s>', '<s>'), 338),
 (('!', '<s>', '<s>'), 242),
 (('qwq', '!', '<s>'), 235),
 (('<s>', '<s>', 'qwq'), 202),
 (('<s>', 'qwq', '!'), 201),
 (('<s>', '<s>', 'and'), 172),
 (('<s>', '<s>', 'i'), 166),
 (('<s>', '<s>', 'but'), 109),
 (('<s>', '<s>', 'the'), 98),
 (('<s>', '<s>', 'it'), 96),
 (('<s>', '<s>', 'we'), 68),
 (('<s>', '<s>', 'he'), 62),
 (('you', 'know', ','), 61),
 (('<s>', '<s>', 'tom'), 60),
 (('<s>', 'tom', 'jarriel'), 60),
 (('<s>', '<s>', 'they'), 54),
 (('<s>', '<s>', 'so'), 52),
 (('<s>', '<s>', 'simon'), 50),
 (('it', '.', '<s>'), 50),
 (('<s>', '<s>', 'that'), 48),
 ((',', 'you', 'know'), 47),
 (('<s>', 'it', "'s"), 45),
 (('<s>', '<s>', 'you'), 44),
 (('<s>', '<s>', 'this'), 42),
 (('<s>', '<s>', 'in'), 42),
 (('tom', 'jarriel', 'voice-over'), 38),
 (('a', 'lot', 'of'), 36),
 (('<s>', '<s>', 'stewart'), 36),
 (('i', 'mean', ','), 36)]

Now that we have a count of all trigrams, we can make predictions by looking for the most common trigram that matches our input. 

In [8]:
def predict_trigram_model(number_results = 3):
    text = input("Enter the start of a sentence, with every word and punctuation mark surrounded by spaces:")
    input_tokens = pad(preprocess(text), 2)
    
    # Find the last 2 tokens in the input
    last_two_tokens = input_tokens[-2:]
    
    # Search our list of all trigrams to find matching trigrams
    matching_trigrams = []
    for item in all_trigrams.items():
        gram = item[0]
        
        # Check if the first and second item in the trigram are a match
        if gram[0] == last_two_tokens[0] and gram[1] == last_two_tokens[1]:
            matching_trigrams.append(item)
    
    # Now, sort the matching trigrams by popularity and return the first `number_results` results
    sorted_matching_trigrams = sorted(matching_trigrams, key=lambda x: x[1], reverse=True)
    top_matching_trigrams = sorted_matching_trigrams[:number_results]
    
    # Last, let's just get the predicted word (the last word of the trigram)
    predictions = [trigram[0][2] for trigram in top_matching_trigrams]
    return predictions

print("Predictions: ", predict_trigram_model())

Enter the start of a sentence, with every word and punctuation mark surrounded by spaces: Hi my name is


Predictions:  ['ana']


### Considerations with n-grams
This is great, but it has one significant issue. If the phrase we enter ends with two words that don't match *any* trigram, then our model has no predictions to make.

One common solution for this is to use a process called **backoff**. With backoff, if our trigram model doesn't find any results, we try to find matching *bigrams* instead. If that fails, we go to *unigrams* (which means we just use the most frequent words). This means that we always get a result, even if it isn't quite as good.

Below is a model that uses backoff and an arbitrary size n-gram to make predictions, putting together everything so far.

In [9]:
n_gram_models = dict()

def create_ngram_model(n = 3):
    padded_tokens = pad(tokens_filtered, n - 1)
    grams = list(ngrams(sequence=padded_tokens, n=n))
    
    all_ngrams = dict()
    for gram in grams:
        if gram in all_ngrams:
            all_ngrams[gram] += 1
        else:
            all_ngrams[gram] = 1
    return all_ngrams

def predict_ngram_model(n = 3, number_results = 3):
    text = input("Enter the start of a sentence, with every word and punctuation mark surrounded by spaces:")
    input_tokens = pad(preprocess(text), n - 1)
    
    while n > 0:
        # Find the last n - 1 tokens in the input
        last_tokens = tuple(input_tokens[-(n-1):])
    
        if not n in n_gram_models:
            n_gram_models[n] = create_ngram_model(n)
        
        matching_ngrams = []
        
        for item in n_gram_models[n].items():
            gram = item[0]
            if gram[:-1] == last_tokens:
                matching_ngrams.append(item)
    
        # Now, sort the matching trigrams by popularity and return the first `number_results` results
        sorted_matching_ngrams = sorted(matching_ngrams, key=lambda x: x[1], reverse=True)
        top_matching_ngrams = sorted_matching_ngrams[:number_results]
        
        # BACKOFF: If there are no results, drop n and try again
        if len(top_matching_ngrams) == 0:
            print("backing off to:", n-1)
            n = n - 1
            continue
    
        # Last, let's just get the predicted word (the last word of the trigram)
        predictions = [gram[0][-1] for gram in top_matching_ngrams]
        return predictions
    return []

predict_ngram_model(4)

Enter the start of a sentence, with every word and punctuation mark surrounded by spaces: What are you


['hearing']